In [1]:
import dask.dataframe as dd
import pandas as pd

In [2]:
from dask.distributed import Client

client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 15.52 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45019,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 15.52 GiB
Comm: tcp://127.0.0.1:37243,Total threads: 2
Dashboard: http://127.0.0.1:41447/status,Memory: 3.88 GiB
Nanny: tcp://127.0.0.1:44651,


In [3]:
cases_dtype = {
    "ddl_case_id": "string[pyarrow]",
    "year": "int16",
    "state_code": "category",
    "dist_code": "category",
    "court_no": "category",
    "cino": "string[pyarrow]",
    'judge_position': "string[pyarrow]",
    "female_defendant": "category",
    "female_adv_def": "category",
    "female_adv_pet": "category",
    "type_name": "Int16",
    "purpose_name": "Int16",
    "disp_name": "Int16",
    "date_of_filing": "string[pyarrow]",
    "date_of_decision": "string[pyarrow]",
    "date_first_list": "string[pyarrow]",
    "date_last_list": "string[pyarrow]",
    "date_next_list": "string[pyarrow]",
}
date_parser = lambda x: pd.to_datetime(str(x), errors='coerce', format="%d-%m-%Y")
case_date_parser = lambda x: pd.to_datetime(str(x), errors='coerce', format="%Y-%m-%d")

In [ ]:
for year in range(2010, 2019):
    cases = dd.read_parquet(f"./Parquet/cases/{year}/", dtype=cases_dtype,
                                       columns=["state_code", "dist_code", "court_no", "date_of_filing", "date_of_decision"],
                                       parse_dates=["date_of_filing", "date_of_decision"])

    finished_cases = cases.dropna(subset=["date_of_filing", "date_of_decision", "state_code", "dist_code", "court_no"])
    finished_cases = finished_cases[
        ((finished_cases["date_of_decision"] - finished_cases["date_of_filing"]).dt.total_seconds() >= 0) &
        ((pd.to_datetime('today') - finished_cases["date_of_decision"]).dt.total_seconds() >= 0) &
        (finished_cases["date_of_filing"].dt.year == year)
        ]


    finished_cases.compute().to_parquet(f"Parquet/finished_cases/{year}/", partition_cols=["state_code", "dist_code", "court_no"], engine="pyarrow")
    print(year)

In [4]:
acts_sections = dd.read_csv("./csv/acts_sections.csv", dtype={
    "ddl_case_id": "string[pyarrow]",
    "act": "Int16",
    "section": "float64",
    "bailable_ipc": "category",
    "number_sections_ipc": "Int16",
    "criminal": "category"
})
acts_sections = acts_sections.set_index("ddl_case_id")

In [5]:
for year in range(2016, 2019):
    cases = dd.read_parquet(f"./Parquet/cases/{year}/", dtype=cases_dtype,
                                       parse_dates=["date_of_filing", "date_of_decision", "date_first_list", "date_last_list", "date_next_list"])

    cases = cases.set_index("ddl_case_id")
    cases = dd.merge(cases, acts_sections, how="inner", on=["ddl_case_id"])
    cases.to_parquet(f"Parquet/cases_acts/{year}/", engine="pyarrow")
    print(year)

2016
2017
2018
